# Experiment 1

### Environment Setup

In [1]:
%%bash -s

# set root folder and get environment
export root="/home/fabric/work"
source $root/env
pwd

/home/fabric/work


In [2]:
# imports
import os
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from fabrictestbed_extensions.fablib.node import Node
from threading import Thread
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import uuid
from collections import defaultdict
from IPython.display import display

# show fablib config
fablib = fablib_manager()             
fablib.show_config()

# constants
root = "/home/fabric/work"

# component names
slice_name = "experiment1"
server_site, client_site = fablib.get_random_sites(count=2, avoid=['AL2S','STAR', 'MAX','TACC'])
print(f"{server_site=}, {client_site=}")

# switch detail
client_switch_name = "client-switch"
server_switch_name = "server-switch"
switch_cores = 2
switch_ram = 8
switch_disk = 100

# host details
client_host_name = "client-host"
server_host_name = "server-host"
image = 'default_ubuntu_20'
server_host_subnet=IPv4Network('10.0.1.0/24')
server_host_addr=IPv4Address('10.0.1.1')
client_host_subnet=IPv4Network('10.0.2.0/24')
client_host_addr=IPv4Address('10.0.2.2')

# network details
net_server_switch_name = "net-server-switch"
net_client_switch_name = "net-client-switch"
net_switch_switch_name = "net-switch-switch"

credmgr_host,cm.fabric-testbed.net
orchestrator_host,orchestrator.fabric-testbed.net
fabric_token,/home/fabric/.tokens.json
project_id,6ce270de-788d-4e07-8bae-3206860a6387
bastion_username,brmclemo_0000026977
bastion_key_filename,/home/fabric/work/ssh/fabric-bastion
bastion_public_addr,bastion-1.fabric-testbed.net
bastion_passphrase,None
slice_public_key_file,/home/fabric/work/ssh/fabric-sliver.pub
slice_private_key_file,/home/fabric/work/ssh/fabric-sliver
fabric_slice_private_key_passphrase,None


server_site='DALL', client_site='MICH'


In [3]:
# some helper functions
__id_counter = defaultdict(int)
def id(name=None):
  global __id_counter
  __id_counter[name] += 1
  return f"{name}-{str(__id_counter[name]).zfill(4)}"

## Setup Network

In [4]:

# create the slice
slice = fablib.new_slice(name=slice_name)

# add server switch
server_switch = slice.add_node(name=server_switch_name, site=server_site, image=image)
server_switch_iface_host = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
server_switch_iface_client_switch = server_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client switch
client_switch = slice.add_node(name=client_switch_name, site=client_site, image=image)
client_switch_iface_host = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]
client_switch_iface_server_switch = client_switch.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add server host
server_host = slice.add_node(name=server_host_name, site=server_site, image=image)
server_host_iface = server_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add client host
client_host = slice.add_node(name=client_host_name, site=client_site, image=image)
client_host_iface = client_host.add_component(model='NIC_Basic', name=id('nic')).get_interfaces()[0]

# add networks
server_switch_net = slice.add_l2network(name=net_server_switch_name, interfaces=[server_switch_iface_host, server_host_iface])
client_switch_net = slice.add_l2network(name=net_client_switch_name, interfaces=[client_switch_iface_host, client_host_iface])
switch_switch_net = slice.add_l2network(name=net_switch_switch_name, interfaces=[server_switch_iface_client_switch, client_switch_iface_server_switch])

# submit slice for creation
slice.submit()



Retry: 8, Time: 214 sec


ID,51147cc1-c77e-4a0c-87db-c86bd0b48673
Name,experiment1
Lease Expiration (UTC),2022-12-01 01:21:32 +0000
Lease Start (UTC),2022-11-30 01:21:33 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Site,Host,Cores,RAM,Disk,Image,Management IP,State,Error
77ce2afd-31b0-49b1-bab3-ad449b7c259a,server-switch,DALL,dall-w1.fabric-testbed.net,2,8,10,default_ubuntu_20,2001:400:a100:3000:f816:3eff:feee:2e5a,Active,
d43f9dcb-de74-494c-8498-385589ca3edf,client-switch,MICH,mich-w1.fabric-testbed.net,2,8,10,default_ubuntu_20,2607:f018:110:11:f816:3eff:fee3:bd85,Active,
1558dcfa-4209-413d-8113-74174b8d6cb7,server-host,DALL,dall-w1.fabric-testbed.net,2,8,10,default_ubuntu_20,2001:400:a100:3000:f816:3eff:fea3:fb01,Active,
f89f59e3-246e-4261-b283-e49de6c72a12,client-host,MICH,mich-w1.fabric-testbed.net,2,8,10,default_ubuntu_20,2607:f018:110:11:f816:3eff:feff:8e0f,Active,


ID,Name,Layer,Type,Site,Gateway,L3 Subnet,State,Error
25275cca-0bec-4813-806a-112d96d67844,net-server-switch,L2,L2Bridge,DALL,None,None,Active,
6f8f4a3d-fd9d-42c8-b762-d2b8c6028800,net-client-switch,L2,L2Bridge,MICH,None,None,Active,
8abd6985-4588-4071-b6ab-fae162c635ee,net-switch-switch,L2,L2STS,None,None,None,Active,



Time to stable 214 seconds
Running post_boot_config ... Time to post boot config 222 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Device
server-switch-nic-0001-p1,server-switch,net-server-switch,0,None,1E:75:8E:3C:A5:DA,ens7
server-switch-nic-0002-p1,server-switch,net-switch-switch,0,None,22:4C:CA:EA:CE:A1,ens8
client-switch-nic-0003-p1,client-switch,net-client-switch,0,None,06:1D:A4:2D:F2:99,ens7
client-switch-nic-0004-p1,client-switch,net-switch-switch,0,None,0A:44:2E:A3:7B:F8,ens8
server-host-nic-0005-p1,server-host,net-server-switch,0,None,22:83:57:67:FB:21,ens7
client-host-nic-0006-p1,client-host,net-client-switch,0,None,0A:C1:6F:AC:81:24,ens7



Time to print interfaces 235 seconds


'51147cc1-c77e-4a0c-87db-c86bd0b48673'

In [5]:
# set up nodes
ip_addrs = dict()
slice = fablib.get_slice(name=slice_name)
switch_available_addrs = list(switch_subnet := IPv4Network("192.168.2.0/24"))[1:]
host_available_addrs = list(host_subnet := IPv4Network("192.168.3.0/24"))[1:]
# any_available_addrs = list(any_subnet := IPv4Network("192.168.3.0/24"))[1:]
switch_addrs = defaultdict(lambda: switch_available_addrs.pop(0))
host_addrs = defaultdict(lambda: host_available_addrs.pop(0))
# any_addrs = defaultdict(lambda: any_available_addrs.pop(0))
for node in slice.get_nodes():
  node_name = node.get_name()
  print(f"{node_name=}")

  # setup
  node.execute("sudo apt update && sudo apt install net-tools && pip install netifaces")
  node.upload_directory('code','~')

  # give ip to all nodes
  for iface in node.get_interfaces():
    iface_name = iface.get_name()
    print(f"{iface_name=}")
    iface.ip_addr_add(addr=switch_addrs[node_name], subnet=switch_subnet)
    iface.ip_link_up()
    
  


node_name='server-switch'
iface_name='server-switch-nic-0001-p1'
iface_name='server-switch-nic-0002-p1'
node_name='client-switch'
iface_name='client-switch-nic-0003-p1'
iface_name='client-switch-nic-0004-p1'
node_name='server-host'
iface_name='server-host-nic-0005-p1'
node_name='client-host'
iface_name='client-host-nic-0006-p1'


In [6]:
# set up l3 networking addresses
for node in slice.get_nodes():
  node_name = node.get_name()
  if 'host' in node_name:
    print(f"{node_name=}")
    # node.ip_route_add(subnet=host_subnet, gateway=switch_addrs[f"{node_name[:-5]}-switch"])
    for iface in node.get_interfaces():
      iface_name = iface.get_name()
      print(f"{iface_name=}")
      iface.ip_addr_add(addr=host_addrs[node_name], subnet=host_subnet)

node_name='server-host'
iface_name='server-host-nic-0005-p1'
node_name='client-host'
iface_name='client-host-nic-0006-p1'


In [10]:
# store values for server
for node in slice.get_nodes():
  node_name = node.get_name()
  if 'switch' in node_name:
    print(f"{node_name=}")
    host_iface = node.get_interface(network_name=f"net-{node_name}")
    network_iface = node.get_interface(network_name=f"net-switch-switch")
    transformer_config = {
      'host': host_iface.get_os_interface(),
      'network': network_iface.get_os_interface(),
      'id': 0,
      'protocols': {
        'ARP': 0x0806,
        'IP': 0x0800,
      }
    }
    transformer_config_json = json.dumps(transformer_config)
    node.execute(f"echo '{transformer_config_json}' >transformer_config.json")
# for net in slice.get_networks():
#   net_name=net.get_name()
#   print(f"{net.get_name()=}")

node_name='server-switch'
node_name='client-switch'


In [8]:
# install and start server switch
raise
slice = fablib.get_slice(name=slice_name)
for net in slice.get_l2networks():
  net.show()

RuntimeError: No active exception to reraise

## Configure Network Components

In [ ]:
subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]
f"{list(subnet)[0]}"

### Configure Hosts

In [ ]:
raise
# config information
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/"

# configure server host
server_host = slice.get_node(name=server_host_name)        
if type(ip_address(server_host.get_management_ip())) is IPv6Address:
    stdout, stderr = server_host.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    print(server_host_name)
    print(f"stdout:\n{stdout}\n")
    print(f"stderr:\n{stderr}\n")
server_host_os_iface = server_host.get_interface(network_name=net_server_switch_name)
server_host_os_iface.ip_addr_add(addr=server_host_addr, subnet=server_host_subnet)
stdout, stderr = server_host.execute(host_config_script)
print(server_host_name)
print(f"stdout:\n{stdout}\n")
print(f"stderr:\n{stderr}\n")

# configure client host
client_host = slice.get_node(name=client_host_name)        
if type(ip_address(client_host.get_management_ip())) is IPv6Address:
    stdout, stderr = client_host.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
    print(client_host_name)
    print(f"stdout:\n{stdout}\n")
    print(f"stderr:\n{stderr}\n")
client_host_os_iface = client_host.get_interface(network_name=net_client_switch_name)
client_host_os_iface.ip_addr_add(addr=client_host_addr, subnet=client_host_subnet)
stdout, stderr = client_host.execute(host_config_script)
print(client_host_name)
print(f"stdout:\n{stdout}\n")
print(f"stderr:\n{stderr}\n")


### Configure Switches

In [ ]:
server_switch = slice.get_node(name=server_switch_name)
server_switch_os_iface = server_switch.get_interface(network_name=net_server_switch_name)
switch_switch_os_iface = server_switch.get_interface(network_name=net_switch_switch_name)
    
if type(ip_address(server_switch.get_management_ip())) is IPv4Address:
    management_ip_type = "IPv4"
else:
    management_ip_type = "IPv6"

file_attributes = server_switch.upload_file(f'{root}/P4_bmv2/scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {server_switch_os_iface.get_os_interface()} {switch_switch_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
s1_config_thread = server_switch.execute_thread(command)
config_threads[server_switch] = s1_config_thread

In [ ]:
client_switch = slice.get_node(name=client_switch_name)
client_switch_os_iface = client_switch.get_interface(network_name=net_client_switch_name)
switch_switch_os_iface = client_switch.get_interface(network_name=net_switch_switch_name)
    
if type(ip_address(client_switch.get_management_ip())) is IPv4Address:
    management_ip_type = "IPv4"
else:
    management_ip_type = "IPv6"

file_attributes = client_switch.upload_file(f'{root}/P4_bmv2/scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {server_switch_os_iface.get_os_interface()} {switch_switch_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
s1_config_thread = client_switch.execute_thread(command)
config_threads[client_switch] = s1_config_thread

In [ ]:
# print the output of all nodes
for node, thread in config_threads.items():
  stdout, stderr = thread.result()
  print(node.get_username())
  print(f"Config thread node {node.get_name()} complete")
  print(f"stdout:\n{stdout}")
  print(f"stderr:\n{stderr}")

## Delete Slice

In [ ]:
raise
slice = fablib.get_slice(name=slice_name)
slice.delete()